In [12]:
from google.colab import drive
drive.mount("/content/drive")
!cp -r /content/drive/MyDrive/IIT-Projects/third-umpire-decision-automation/.creds/.aws ~/.aws

In [2]:
!pip install -qq sagemaker ultralytics onnx onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 748.0/748.0 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 562.2/562.2 kB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 15.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.1/175.1 kB 7.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.7/201.7 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 k

In [16]:
# raw model paths
object_detect_model_path = "/content/drive/MyDrive/IIT-Projects/third-umpire-decision-automation/backend/training-jobs/cricket-object-detect17/weights/best.pt"
batsmen_segment_model_path = "/content/drive/MyDrive/IIT-Projects/third-umpire-decision-automation/backend/training-jobs/batsmen-segmentation4/weights/best.pt"
wicket_classification_model_path = "/content/drive/MyDrive/IIT-Projects/third-umpire-decision-automation/backend/training-jobs/batsmen-segmentation4/weights/best.pt"
raw_model_paths = (object_detect_model_path, batsmen_segment_model_path, wicket_classification_model_path)

# onnx upload paths
object_detect_onnx_s3_uri = "s3://third-umpire-decision-automation-osura/model-weights/yolov8_cricket_object_detect.onnx"
batsmen_segment_onnx_s3_uri = "s3://third-umpire-decision-automation-osura/model-weights/yolov8_batsmen_image_segment.onnx"
wicket_classification_onnx_s3_uri = "s3://third-umpire-decision-automation-osura/model-weights/yolov8_wicket_classification.onnx"
onnx_s3_uris = (object_detect_onnx_s3_uri, batsmen_segment_onnx_s3_uri, wicket_classification_onnx_s3_uri)

In [17]:
from ultralytics import YOLO
onnx_export_paths = []
for model_path in raw_model_paths:
    model = YOLO(model_path)  # load a custom trained
    onnx_path = model.export(format='onnx')
    onnx_export_paths.append(onnx_path)

Ultralytics YOLOv8.0.90 🚀 Python-3.10.11 torch-2.0.0+cu118 CPU
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from /content/drive/MyDrive/IIT-Projects/third-umpire-decision-automation/backend/training-jobs/cricket-object-detect17/weights/best.pt with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 6, 8400) (23.4 MB)

ONNX: starting export with onnx 1.13.1 opset 17...
ONNX: export success ✅ 1.1s, saved as /content/drive/MyDrive/IIT-Projects/third-umpire-decision-automation/backend/training-jobs/cricket-object-detect17/weights/best.onnx (11.7 MB)

Export complete (1.7s)
Results saved to /content/drive/MyDrive/IIT-Projects/third-umpire-decision-automation/backend/training-jobs/cricket-object-detect17/weights
Predict:         yolo predict task=detect model=/content/drive/MyDrive/IIT-Projects/third-umpire-decision-automation/backend/training-jobs/cricket-object-detect17/weights/best.onnx imgsz=640 
Validate:        yolo val task=d

============= Diagnostic Run torch.onnx.export version 2.0.0+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



Ultralytics YOLOv8.0.90 🚀 Python-3.10.11 torch-2.0.0+cu118 CPU
YOLOv8n-seg summary (fused): 195 layers, 3258259 parameters, 0 gradients, 12.0 GFLOPs

PyTorch: starting from /content/drive/MyDrive/IIT-Projects/third-umpire-decision-automation/backend/training-jobs/batsmen-segmentation4/weights/best.pt with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 37, 8400), (1, 32, 160, 160)) (25.3 MB)

ONNX: starting export with onnx 1.13.1 opset 17...
ONNX: export success ✅ 2.1s, saved as /content/drive/MyDrive/IIT-Projects/third-umpire-decision-automation/backend/training-jobs/batsmen-segmentation4/weights/best.onnx (12.6 MB)

Export complete (3.3s)
Results saved to /content/drive/MyDrive/IIT-Projects/third-umpire-decision-automation/backend/training-jobs/batsmen-segmentation4/weights
Predict:         yolo predict task=segment model=/content/drive/MyDrive/IIT-Projects/third-umpire-decision-automation/backend/training-jobs/batsmen-segmentation4/weights/best.onnx imgsz=640 
Validate: 

============= Diagnostic Run torch.onnx.export version 2.0.0+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



Ultralytics YOLOv8.0.90 🚀 Python-3.10.11 torch-2.0.0+cu118 CPU
YOLOv8n-seg summary (fused): 195 layers, 3258259 parameters, 0 gradients, 12.0 GFLOPs

PyTorch: starting from /content/drive/MyDrive/IIT-Projects/third-umpire-decision-automation/backend/training-jobs/batsmen-segmentation4/weights/best.pt with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 37, 8400), (1, 32, 160, 160)) (25.3 MB)

ONNX: starting export with onnx 1.13.1 opset 17...
ONNX: export success ✅ 1.3s, saved as /content/drive/MyDrive/IIT-Projects/third-umpire-decision-automation/backend/training-jobs/batsmen-segmentation4/weights/best.onnx (12.6 MB)

Export complete (2.2s)
Results saved to /content/drive/MyDrive/IIT-Projects/third-umpire-decision-automation/backend/training-jobs/batsmen-segmentation4/weights
Predict:         yolo predict task=segment model=/content/drive/MyDrive/IIT-Projects/third-umpire-decision-automation/backend/training-jobs/batsmen-segmentation4/weights/best.onnx imgsz=640 
Validate: 

============= Diagnostic Run torch.onnx.export version 2.0.0+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [15]:
from sagemaker.s3 import S3Uploader

for onnx_path, s3_uri in zip(onnx_export_paths, onnx_s3_uris):
    S3Uploader.upload(onnx_path, s3_uri)